In [1]:
import numpy as np

In [2]:
X_train = np.load('Datasets/Fashion_MNIST/X_train.npy')
X_test = np.load('Datasets/Fashion_MNIST/X_test.npy')
y_train = np.load('Datasets/Fashion_MNIST/y_train.npy')
y_test = np.load('Datasets/Fashion_MNIST/y_test.npy')

In [3]:
X_train.shape

(60000, 784)

In [4]:
y_train.shape

(60000,)

In [5]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
encoder = OneHotEncoder(sparse=False)

In [8]:
encoder.fit(y_train.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [9]:
y_train = encoder.transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

### Training

In [10]:
import tensorflow as tf

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
def dnn(X, output_dim, layers, drop_rate, reuse, is_training):
    with tf.variable_scope('DenselyConnected', reuse=reuse):
        regularizer = tf.contrib.layers.l2_regularizer(scale=regularization_rate)
        current_layer = X
        for units in layers:
            l = tf.layers.dense(current_layer,
                                units,
                                activation=tf.nn.tanh,
                                kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                kernel_regularizer=regularizer)
            
            current_layer = tf.layers.dropout(l, rate=drop_rate, training=is_training)
        
        output = tf.layers.dense(current_layer, output_dim)
    return output

In [12]:
def model(X, y, layers, learning_rate, drop_rate):
    train_logits = dnn(X, y.shape[1].value, layers, drop_rate, reuse=False, is_training=True)
    test_logits = dnn(X, y.shape[1].value, layers, drop_rate, reuse=True, is_training=False)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=train_logits, labels=y))
    loss += tf.losses.get_regularization_loss('DenselyConnected')
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss)

    
    train_pred = tf.equal(tf.argmax(train_logits,1), tf.argmax(y,1))
    train_accuracy = tf.reduce_mean(tf.cast(train_pred,tf.float32))
    
    test_pred = tf.equal(tf.argmax(test_logits,1), tf.argmax(y,1))
    test_accuracy = tf.reduce_mean(tf.cast(test_pred,tf.float32))
    return train_op, loss, train_accuracy, test_accuracy

In [13]:
n_features = X_train.shape[1]
n_labels = y_train.shape[1]

In [18]:
Batch_size = 256
LR = 0.0005
epochs = 100
drop_rate = 0.0
regularization_rate = 0.0
Layers = [256, 128, 64, 32]

In [19]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, n_features], name='features')
y = tf.placeholder(tf.float32, shape=[None, n_labels], name='labels')

with tf.Session() as sess:
    train_op, loss, train_accuracy, test_accuracy = model(X, y, Layers, LR, drop_rate)
    sess.run(tf.global_variables_initializer())

    for epoch in range(1, epochs+1):
        for X_batch, y_batch in get_next_batch(X_train, y_train, Batch_size):
            _, l, train_acc = sess.run([train_op, loss, train_accuracy], {X: X_batch, y: y_batch})
            
        if epoch%10 == 0:
            for X_batch, y_batch in get_next_batch(X_test, y_test, Batch_size):
                test_acc, = sess.run([test_accuracy], {X: X_batch, y: y_batch})
            print("Epoch:", epoch, " Loss:", l, " Train accuracy:", train_acc, " Test accuracy:", test_acc)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
Epoch: 10  Loss: 0.67562664  Train accuracy: 0.734375  Test accuracy: 0.796875
Epoch: 20  Loss: 0.44630224  Train accuracy: 0.84765625  Test accuracy: 0.796875
Epoch: 30  Loss: 0.5172331  Train accuracy: 0.8203125  Test accuracy: 0.80859375
Epoch: 40  Loss: 0.49093974  Train accuracy: 0.80078125  Test accuracy: 0.81640625
Epoch: 50  Loss: 0.557464  Train accuracy: 0.79296875  Test accuracy: 0.77734375
Epoch: 60  Loss: 0.415496  Train accuracy: 0.8359375  Test accuracy: 0.85546875
Epoch: 70  Loss: 0.47201005  Train accuracy: 0.8203125  Test accuracy: 0.8203125
Epoch: 80  Loss: 0.55239075  Train accuracy: 0.76171875  Test accuracy: 0.83203125
Epoch: 90  Loss: 0.4115301  Train accuracy: 0.84375  Test accuracy: 0.796875
Epoch: 100  Loss: 0.4304093  Train accuracy: 0.8515625  Test accuracy: 0.82421875
